In [21]:
import numpy as np
import keras
from keras import backend as K
from keras.layers.core import Dense
from keras.optimizers import Adam,RMSprop
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.callbacks import TensorBoard, ModelCheckpoint,EarlyStopping
from keras.applications import imagenet_utils,MobileNet
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
MobileNet=MobileNet(weights='imagenet',include_top=False,input_shape=(224,224,3))

In [23]:
for layer in MobileNet.layers:
    layer.trainable=False

In [24]:
def fullyConn(bottom_Model):
    top_model=bottom_Model.output
    top_model=keras.layers.GlobalAveragePooling2D()(top_model)
    top_model=keras.layers.Dense(1024,activation='relu')(top_model)
    top_model=keras.layers.Dense(1024,activation='relu')(top_model)
    top_model=keras.layers.Dense(512,activation='relu')(top_model)
    top_model=keras.layers.Dense(2,activation='softmax')(top_model)
    
    return top_model
    

In [25]:
new_mod=fullyConn(MobileNet)

model=Model(inputs=MobileNet.input,outputs=new_mod)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [26]:
train_path="C:/Users/BEVAN/PycharmProjects/MaskDetection/Data"

In [27]:
train_datagen=ImageDataGenerator(rescale=1./255)

In [28]:
validation_datagen=ImageDataGenerator(rescale=1./255)

In [29]:
batch_size=16

In [30]:
train_generator=train_datagen.flow_from_directory(train_path,target_size=(224,224),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')
validation_generator=validation_datagen.flow_from_directory(train_path,
                                                           target_size=(224,224),
                                                           batch_size=batch_size,
                                                           class_mode='categorical')

Found 1376 images belonging to 2 classes.
Found 1376 images belonging to 2 classes.


In [31]:
checkpoint=ModelCheckpoint("mask_recog.h5",monitor="val_loss",mode="min",save_best_only=True,verbose=1)
earlystop=EarlyStopping(monitor="val_loss",min_delta=0,patience=3,verbose=1,restore_best_weights=True)
callbacks=[checkpoint,earlystop]

In [32]:
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['accuracy'])

In [33]:
history=model.fit_generator(train_generator,steps_per_epoch=688/16,epochs=30,
                           callbacks=callbacks,validation_data=validation_generator,validation_steps=688/16)

Epoch 1/30
43/43 [==============================] - 15s 354ms/step - loss: 1.0026 - accuracy: 0.8241 - val_loss: 0.0696 - val_accuracy: 0.9869

Epoch 00001: val_loss improved from inf to 0.06958, saving model to mask_recog.h5
Epoch 2/30
43/43 [==============================] - 14s 322ms/step - loss: 0.2669 - accuracy: 0.9172 - val_loss: 0.0032 - val_accuracy: 0.9927

Epoch 00002: val_loss improved from 0.06958 to 0.00318, saving model to mask_recog.h5
Epoch 3/30
43/43 [==============================] - 14s 324ms/step - loss: 0.1285 - accuracy: 0.9622 - val_loss: 0.0013 - val_accuracy: 0.9942

Epoch 00003: val_loss improved from 0.00318 to 0.00132, saving model to mask_recog.h5
Epoch 4/30
43/43 [==============================] - 14s 323ms/step - loss: 0.1827 - accuracy: 0.9549 - val_loss: 0.0025 - val_accuracy: 0.9956

Epoch 00004: val_loss did not improve from 0.00132
Epoch 5/30
43/43 [==============================] - 16s 366ms/step - loss: 0.1165 - accuracy: 0.9724 - val_loss: 8.5022